In [1]:
import os 
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import json

In [3]:
#'random_32', 'lru_32random_64', 'lru_64random_128', 'lru_128random_256',
#       'lru_256random_512', 'lru_512random_1024', 'lru_1024random_2048',
#      'lru_2048random_4096', 'lru_4096random_8192', 'lru_8192'
o = []
for file in os.listdir('StatCache/output'):
    x = pd.read_csv('StatCache/output/'+file)
    s = []
    print(file)
    for col in x.columns:
        print(f"{x[col][1:].var():.2f}",end=" ")
        s.append(x[col].var())
    o.append(s)
    print()


400.perlbench-41B_1024.csv
0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 
400.perlbench-50B_1024.csv
0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 
401.bzip2-226B_1024.csv
0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 
401.bzip2-277B_1024.csv
0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 
401.bzip2-38B_1024.csv
0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 
401.bzip2-7B_1024.csv
0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 
403.gcc-16B_1024.csv
0.02 0.07 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 
403.gcc-17B_1024.csv
0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 
403.gcc-48B_1024.csv
0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 
410.bwaves-1963B_1024.csv
0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 
410.bwaves-2097B_1024.csv
0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 
410.bwaves-945B_1024.csv
0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 
416.gamess-875B_1024.csv
0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 
429.mcf-184B_1024.csv
0.0

0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 
607.cactuBSSN_s-4004B_1024.csv
0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 
607.cactuBSSN_s-4248B_1024.csv
0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 
619.lbm_s-2676B_1024.csv
0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 
619.lbm_s-2677B_1024.csv
0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 
619.lbm_s-3766B_1024.csv
0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 
619.lbm_s-4268B_1024.csv
0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 
620.omnetpp_s-141B_1024.csv
0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 
620.omnetpp_s-874B_1024.csv
0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 
621.wrf_s-575B_1024.csv
0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 
621.wrf_s-6673B_1024.csv
0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 
621.wrf_s-8065B_1024.csv
0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 
621.wrf_s-8100B_1024.csv
0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 
623.xalancbmk_s-10B_

In [4]:
x = pd.read_csv('StatCache/output/619.lbm_s-2676B_1024.csv')
print(x['random_32'][1:])

0.220545  0.214247  0.206815  0.214247  0.173619  0.210610  0.135296  0.095419    0.112657
0.208341  0.202290  0.195648  0.202290  0.164282  0.202290  0.128035  0.088443    0.106738
0.223352  0.217072  0.209927  0.217072  0.177890  0.197628  0.140300  0.100489    0.117539
0.206571  0.201425  0.192841  0.198349  0.162879  0.176894  0.129011  0.094074    0.108568
0.195038  0.189370  0.183077  0.189370  0.153420  0.189370  0.119064  0.080893    0.098927
                                                                                    ...   
0.270767  0.262996  0.255084  0.262996  0.216274  0.253613  0.169591  0.119506    0.141216
0.262468  0.254912  0.246785  0.254912  0.207792  0.254156  0.161902  0.113193    0.134747
0.261919  0.255693  0.244344  0.255693  0.206083  0.223576  0.162879  0.117122    0.136517
0.264482  0.256840  0.248738  0.256840  0.210355  0.248196  0.165198  0.117333    0.138042
0.268326  0.260910  0.252949  0.260910  0.214443  0.249661  0.168432  0.118209    0.140544

In [5]:
for i in o:
    if max(i) > 0.01:
        print(i)

[0.020670481332396785, 0.06783952178994157, 0.011538729746518522, 0.011587651827721302, 0.011446601554703126, 0.011399363752124862, 0.0109001738043167, 0.010990781195176155, 0.009698381375518697, 0.01007272402453609]
[0.014521965359696007, 0.014158829144972573, 0.016473073404645006, 0.015260738370445772, 0.020995226338761355, 0.018003671863177088, 0.018748804148717563, 0.014947412737219237, 0.015918072449312853, 0.012416090160902029]
[0.010179693106465227, 0.010458643508898352, 0.016716764938013298, 0.015755783906870116, 0.026309289124680352, 0.02293143115951556, 0.024349038944295912, 0.018073028202536416, 0.01956424200193582, 0.015032538411328856]
[0.02494631559933662, 0.024677131999273492, 0.02407481831109748, 0.025274216671919036, 0.017833862599318132, 0.025018763570237374, 0.005579757499233006, 0.014954221318055731, 0.0014375114402149641, 0.0015260769875627075]
[0.00962084772765291, 0.009838660218268838, 0.012173128986773375, 0.011841517117487741, 0.014483423599988393, 0.0136891448

In [1]:
import cv2

In [11]:
# METHOD #3: ROLL YOUR OWN
def chi2_distance(histA, histB, eps = 1e-10):
    # compute the chi-squared distance
    d = 0.5 * np.sum([((a - b) ** 2) / (a + b + eps)
        for (a, b) in zip(histA, histB)])
    # return the chi-squared distance
    return d

In [2]:
keep_dict = {}
with open('keep_rds.txt', 'r') as file:
     keep_dict = json.loads(file.read())
for file in os.listdir('dataset/input'):
    if file in keep_dict:
        continue
    print(file)
    x = pd.read_csv('dataset/input/'+file)
    prev_min = 0
    tmp = []
    keep_list = []
    for i in range(len(x)):
        if i==0: continue
        d = np.zeros((x['0'][i]))
        for col in x.columns[1:]:
            if x[col][i] == 0:
                continue
            s = np.full((int(round(x[col][i]/16))),int(col))
            d = np.append(d, s)
        #d = np.array(x.iloc[i]) + 1
        checker = True
        tmp_ctr = 0
        if not len(d):
            d = [0]
        for other_row in tmp[::-1]:
            tmp_ctr += 1
            rr = stats.kstest(other_row, d)
            # rr = stats.chisquare(other_row, d)
            # f = np.array([other_row, d])
            # rr = stats.chi2_contingency(f)
            # If we have a histogram that matches then drop this one
            if rr.pvalue < 0.05:
                checker = False
                break
        if checker:
            tmp.append(d)
            keep_list.append(1)
        else:
            keep_list.append(0)
    fnam = file[:-9]
    keep_dict[file] = keep_list.copy()
    print(fnam, sum(keep_dict[file]))
    with open('keep_rds.txt', 'w') as file_1:
        file_1.write(json.dumps(keep_dict)) # use `json.loads` to do the reverse

623.xalancbmk_s-10B_1024.csv
623.xalancbmk_s-10B 2
623.xalancbmk_s-165B_1024.csv
623.xalancbmk_s-165B 4
623.xalancbmk_s-202B_1024.csv
623.xalancbmk_s-202B 1
623.xalancbmk_s-325B_1024.csv
623.xalancbmk_s-325B 1
623.xalancbmk_s-592B_1024.csv
623.xalancbmk_s-592B 16
623.xalancbmk_s-700B_1024.csv
623.xalancbmk_s-700B 2
625.x264_s-12B_1024.csv


KeyboardInterrupt: 

In [23]:
print(keep_dict.keys())

dict_keys(['400.perlbench-41B_1024.csv', '400.perlbench-50B_1024.csv', '401.bzip2-226B_1024.csv', '401.bzip2-277B_1024.csv', '401.bzip2-38B_1024.csv', '401.bzip2-7B_1024.csv', '403.gcc-16B_1024.csv', '403.gcc-17B_1024.csv', '403.gcc-48B_1024.csv', '410.bwaves-1963B_1024.csv', '410.bwaves-2097B_1024.csv', '410.bwaves-945B_1024.csv', '416.gamess-875B_1024.csv', '429.mcf-184B_1024.csv', '429.mcf-192B_1024.csv', '429.mcf-217B_1024.csv', '429.mcf-22B_1024.csv', '429.mcf-51B_1024.csv', '433.milc-127B_1024.csv', '433.milc-274B_1024.csv', '433.milc-337B_1024.csv', '434.zeusmp-10B_1024.csv', '435.gromacs-111B_1024.csv', '435.gromacs-134B_1024.csv', '435.gromacs-226B_1024.csv', '435.gromacs-228B_1024.csv', '436.cactusADM-1804B_1024.csv', '437.leslie3d-134B_1024.csv', '437.leslie3d-149B_1024.csv', '437.leslie3d-232B_1024.csv', '437.leslie3d-265B_1024.csv', '437.leslie3d-271B_1024.csv', '437.leslie3d-273B_1024.csv', '444.namd-120B_1024.csv', '444.namd-166B_1024.csv', '444.namd-23B_1024.csv', '444.

In [8]:
ctr = 0
for fname in keep_dict.keys():
    print(f'{sum(keep_dict[fname])/len(keep_dict[fname]):.3f}, {sum(keep_dict[fname])} / {len(keep_dict[fname])}')

0.004, 3 / 749
0.001, 1 / 749
0.011, 8 / 749
0.007, 5 / 749
0.015, 11 / 749
0.005, 4 / 749
0.003, 2 / 749
0.004, 3 / 749
0.068, 51 / 749
0.085, 64 / 749
0.080, 60 / 749
0.462, 346 / 749
0.001, 1 / 749
0.001, 1 / 749
0.001, 1 / 749
0.032, 24 / 749
0.005, 4 / 749
0.001, 1 / 749
0.019, 14 / 749
0.008, 6 / 749
0.001, 1 / 749
0.009, 7 / 749
0.003, 2 / 749
0.001, 1 / 749
0.001, 1 / 749
0.335, 251 / 749
0.475, 356 / 749
0.004, 3 / 749
0.001, 1 / 749
0.005, 4 / 749
0.004, 3 / 749
0.003, 2 / 749
0.013, 10 / 749
0.009, 7 / 749
0.008, 6 / 749
0.004, 3 / 749
0.001, 1 / 749
0.007, 5 / 749
0.007, 5 / 749
0.017, 13 / 749
0.109, 82 / 749
0.044, 33 / 749
0.101, 76 / 749
0.089, 67 / 749
0.003, 2 / 749
0.001, 1 / 749
0.001, 1 / 749
0.246, 184 / 749
0.194, 145 / 749
0.422, 316 / 749
0.489, 366 / 749
0.001, 1 / 749
0.001, 1 / 749
0.063, 47 / 749
0.059, 44 / 749
0.028, 21 / 749
0.116, 87 / 749
0.013, 10 / 749
0.040, 30 / 749
0.055, 41 / 749
0.003, 2 / 749
0.039, 29 / 749
0.001, 1 / 749
0.013, 10 / 749
0.052

In [24]:
with open('keep_rds.txt', 'w') as file:
     file.write(json.dumps(keep_dict)) # use `json.loads` to do the reverse

In [66]:
ff = np.full((3), 7)
ffc = np.full((11), 6)
ff = np.append(ff, ffc)
print(ff)

[7 7 7 6 6 6 6 6 6 6 6 6 6 6]


In [62]:
with open('keep_rds.txt', 'r') as file:
     tmpx = json.loads(file.read()) # use `json.loads` to do the reverse

In [63]:
print(tmpx.keys())

dict_keys(['400.perlbench-41B_1024.csv', '400.perlbench-50B_1024.csv', '401.bzip2-226B_1024.csv', '401.bzip2-277B_1024.csv', '401.bzip2-38B_1024.csv', '401.bzip2-7B_1024.csv', '403.gcc-16B_1024.csv', '403.gcc-17B_1024.csv', '403.gcc-48B_1024.csv', '410.bwaves-1963B_1024.csv', '410.bwaves-2097B_1024.csv', '410.bwaves-945B_1024.csv', '416.gamess-875B_1024.csv', '429.mcf-184B_1024.csv', '429.mcf-192B_1024.csv', '429.mcf-217B_1024.csv', '429.mcf-22B_1024.csv', '429.mcf-51B_1024.csv', '433.milc-127B_1024.csv', '433.milc-274B_1024.csv', '433.milc-337B_1024.csv', '434.zeusmp-10B_1024.csv', '435.gromacs-111B_1024.csv', '435.gromacs-134B_1024.csv', '435.gromacs-226B_1024.csv', '435.gromacs-228B_1024.csv', '436.cactusADM-1804B_1024.csv', '437.leslie3d-134B_1024.csv', '437.leslie3d-149B_1024.csv', '437.leslie3d-232B_1024.csv', '437.leslie3d-265B_1024.csv', '437.leslie3d-271B_1024.csv', '437.leslie3d-273B_1024.csv', '444.namd-120B_1024.csv', '444.namd-166B_1024.csv', '444.namd-23B_1024.csv', '444.